Moodle XML Question Generator, (C) 2019-2020 Kosaku Nagasaka (Kobe University)

# moodle_xqgでのネットからの問題の作り方

Moodle XML Question Generator の Python 版である moodle_xqg で，ネットのリソースを使って問題を作る方法を説明します。

## パッケージの読み込み

**moodle_xqg.core**は必ず必要です。**moodle_xqgb.qbank.common**は数式のLaTeX化など，問題作成に必要なものが含まれているので，読み込んだほうがよいです。

In [1]:
import moodle_xqg.core as mxqg
import moodle_xqg.qbank.common as mxqg_common

次のパッケージは作られる問題に依存しますが，多くの場合，乱数生成と数式処理のために，必要となると思います。今回は，SVGへの出力やその可能のために，**io**と**re**を必要とします。

In [2]:
import random
import re
import urllib

## 青空文庫からの文章を使った問題のサンプル

問題生成器は，**Question**の小クラスとして定義する必要があります。必要なメソッドは，**question_generate**と**correct_answers_generate**と**incorrect_answers_generate**と**question_text**と**answer_text**の5つです。

問題生成器は，**Quiz.quiz_identifier**に同じ問題であるかを判定可能なデータを必ず格納してください。この値でもって問題の区別をするので，格納していない場合，最初の1問しか自動生成されなくなってしまいます。 

青空文庫の文章は， **urllib** を使って取り込み， **re** などで整形しています。

In [3]:
class aozora_galaxy_night(mxqg.Question):
    def __init__(self, show_lines=5, safty_lines=2):
        self.show_lines = show_lines
        self.safty_lines = safty_lines
        universe_html = urllib.request.urlopen(
            r'https://www.aozora.gr.jp/cards/000081/files/456_15050.html').read()
        body_text = universe_html.decode('shift_jis')
        main_text=re.findall(r'<div class="main_text">.*<div class="bibliographical_information">',body_text, re.DOTALL)[0]
        lines = re.split(r'\r\n',main_text)
        lines_wo_ruby = [re.sub(r'<ruby><rb>([^<]*)</rb><rp>（</rp><rt>[^<]*</rt><rp>）</rp></ruby>',r'\1',line) for line in lines]
        self.main_lines = []
        for line in lines_wo_ruby:
            if not re.match(r'<.*', line):
                self.main_lines.append(line)
        self.main_lines = [re.sub(r'<[^>]*>','',line.replace('\u3000','')) 
                           for line in self.main_lines]
        self.num_of_lines = len(self.main_lines)
    def question_generate(self, _quiz_number=0):
        quiz = mxqg.Quiz(name='銀河鉄道の夜より', quiz_number=_quiz_number, lang='ja')
        quiz.data = random.randint(self.show_lines//2, self.num_of_lines-self.show_lines//2-1)        
        quiz.quiz_identifier = hash(quiz.data)
        # 正答の選択肢の生成
        ans = { 'fraction': 100, 'data': self.main_lines[quiz.data] }
        quiz.answers.append(ans)
        return quiz        
    def correct_answers_generate(self, quiz, size=1):
        # 正答を個別には作らないので何もしない
        pass
    def incorrect_answers_generate(self, quiz, size=4):
        answers = []
        ans = { 'fraction': 0, 'feedback': '前後のつながりを確認しましょう。' }
        # 傾き符号違い
        target_ids = list(set(range(0,quiz.data - self.show_lines//2 - self.safty_lines)).union(
                    set(range(quiz.data + self.show_lines//2 + self.safty_lines + 1, self.num_of_lines))))
        while len(answers) < size:
            idx_p = random.randint(0,len(target_ids))
            idx = target_ids[idx_p]
            del target_ids[idx_p]
            ans['data'] = self.main_lines[idx]
            answers.append(dict(ans))            
        if len(answers) >= size:
            return random.sample(answers,k=size)
        return answers    
    def question_text(self, quiz):
        qtext = '<p>次の文章は，銀河鉄道の夜の一部分です。空欄に最も適すると思われる文を選択してください。</p>'
        qtext += '<div style="padding: 1em">'
        for idx in range(quiz.data - self.show_lines//2, quiz.data):
            qtext += '<p>' + self.main_lines[idx] + '</p>' 
        qtext += '<p>________（空欄）________</p>'
        for idx in range(quiz.data + 1, quiz.data + self.show_lines//2 +1):
            qtext += '<p>' + self.main_lines[idx] + '</p>' 
        qtext += '</div>'
        return qtext
    def answer_text(self, ans):
        return ans['data']

問題の自動生成は，**generate**を使いますが，それに先立ち，問題生成器のインスタンスを生成しておく必要があります。作成する問題数は，*generate*に**size=100**のような形で指定します（無指定時は，10個生成されます）。

In [4]:
agn = aozora_galaxy_night()
agn_quizzes = mxqg.generate(agn, category='青空文庫の問題サンプル')

生成結果は，**Quizzes**のインスタンスとなっており，次のようにプレビューができます。

In [5]:
agn_quizzes.preview(size=2)

このようにして生成した結果は，**save**を使ってMoodleにインポート可能なファイルに保存することができます。

In [6]:
agn_quizzes.save('aozora_galaxy_night_in_japanese.xml')

実際にインポートして確認してみてください。なお，問題整理番号の表示が不要な場合は，**show_quiz_number=False**をオプションとして*save*に付けてください。

## 形態素解析を使った問題のサンプル

形態素解析は **janome** を使って行います。

In [7]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import *

穴埋めにする名詞の選択肢を頻度に応じて採用するためと，実際に穴埋め部分を作るための分かち書きなどにするためで，都合2つの解析器を用意しています。

In [8]:
class aozora_galaxy_night_noun(mxqg.Question):
    def __init__(self):
        universe_html = urllib.request.urlopen(
            r'https://www.aozora.gr.jp/cards/000081/files/456_15050.html').read()
        body_text = universe_html.decode('shift_jis')
        main_text=re.findall(r'<div class="main_text">.*<div class="bibliographical_information">',body_text, re.DOTALL)[0]
        lines = re.split(r'\r\n',main_text)
        lines_wo_ruby = [re.sub(r'<ruby><rb>([^<]*)</rb><rp>（</rp><rt>[^<]*</rt><rp>）</rp></ruby>',r'\1',line) for line in lines]
        self.main_lines = []
        for line in lines_wo_ruby:
            if not re.match(r'<.*', line):
                self.main_lines.append(line)
        self.main_lines = [re.sub(r'<[^>]*>','',line.replace('\u3000','')) 
                           for line in self.main_lines]
        self.num_of_lines = len(self.main_lines)
        self.main_text = ''
        for line in self.main_lines:
            self.main_text += line
        self.ctoken_filters = [CompoundNounFilter(), POSKeepFilter('名詞'), POSStopFilter(["名詞,代名詞","名詞,非自立","名詞,数"]), TokenCountFilter()]
        self.canalyzer = Analyzer(token_filters=self.ctoken_filters)
        tokens = self.canalyzer.analyze(self.main_text)
        self.nouns = []
        self.weights = []
        for token in tokens:
            self.nouns.append(token[0])
            self.weights.append(token[1])
        self.wtoken_filters = [CompoundNounFilter()]
        self.wanalyzer = Analyzer(token_filters=self.wtoken_filters)
    def _question_generate(self):
        line_id = random.randint(0, self.num_of_lines - 1)
        tokens = self.canalyzer.analyze(self.main_lines[line_id])
        words = []
        for token in tokens:
            words.append(token[0])
        if len(words) == 0:
            return self._question_generate()
        target_noun = random.choice(words)
        return [line_id, target_noun]
    def question_generate(self, _quiz_number=0):
        quiz = mxqg.Quiz(name='銀河鉄道の夜より穴埋め', quiz_number=_quiz_number, lang='ja')
        quiz.data = self._question_generate()
        quiz.quiz_identifier = hash('{}+{}'.format(quiz.data[0], quiz.data[1]))
        # 正答の選択肢の生成
        ans = { 'fraction': 100, 'data': quiz.data[1] }
        quiz.answers.append(ans)
        return quiz        
    def correct_answers_generate(self, quiz, size=1):
        # 正答を個別には作らないので何もしない
        pass
    def incorrect_answers_generate(self, quiz, size=4):
        answers = []
        ans = { 'fraction': 0 }
        # 生成
        while len(answers) < size:
            wrong_noun = random.choices(self.nouns, self.weights)[0]
            if wrong_noun != quiz.data[1]:
                ans['data'] = wrong_noun
                answers.append(dict(ans))            
        if len(answers) >= size:
            return random.sample(answers,k=size)
        return answers    
    def question_text(self, quiz):
        qtext = '<p>次の文章は，銀河鉄道の夜の一部分です。空欄に最も適すると思われる語句を選択してください。</p>'
        qtext += '<div style="padding: 1em">'
        tokens = self.wanalyzer.analyze(self.main_lines[quiz.data[0]])
        for token in tokens:
            if token.surface == quiz.data[1]:
                qtext += '【空欄】'
            else:
                qtext += token.surface
        qtext += '</div>'
        return qtext
    def answer_text(self, ans):
        return ans['data']

問題の自動生成は，**generate**を使いますが，それに先立ち，問題生成器のインスタンスを生成しておく必要があります。作成する問題数は，*generate*に**size=100**のような形で指定します（無指定時は，10個生成されます）。

In [9]:
agnn = aozora_galaxy_night_noun()
agnn_quizzes = mxqg.generate(agnn, category='形態素解析の問題サンプル')

生成結果は，**Quizzes**のインスタンスとなっており，次のようにプレビューができます。

In [10]:
agnn_quizzes.preview(size=2)

このようにして生成した結果は，**save**を使ってMoodleにインポート可能なファイルに保存することができます。

In [11]:
agnn_quizzes.save('aozora_galaxy_night_noun_in_japanese.xml')

実際にインポートして確認してみてください。なお，問題整理番号の表示が不要な場合は，**show_quiz_number=False**をオプションとして*save*に付けてください。